In [5]:
import os
import pandas as pd
import cufflinks as cf
from analitics import Profit, Inflation
from storage import FStruct, FStorage, INFLATION, SECURITIES, DIVIDENDS, TRADE_HISTORY, MARKETDATA, DIVIDENDS_PROCESSED

cf.set_config_file(offline=True)
cf.go_offline()

tax_value = 0.13
broker_commission = 0.07+0.01
root_dir = os.path.abspath("")

fstruct = FStruct(root_dir)
fstorage = FStorage(root_dir)
inflation = Inflation(root_dir)
profit = Profit(tax_value, broker_commission, inflation)

In [21]:
def calc_rate(sec_id, sale_year, sale_month, price_field):
    sale_month_start = pd.Timestamp(sale_year, sale_month, 1)
    sale_month_finish = pd.Timestamp(sale_year, sale_month, sale_month_start.days_in_month)

    hist = fstorage.open_data(TRADE_HISTORY, sec_id, index_col="TRADEDATE").sort_values("TRADEDATE", ascending = True)[[price_field]]
    divs = fstorage.open_data(DIVIDENDS_PROCESSED, sec_id, index_col="t2date").sort_values("t2date", ascending = True)[["value"]]

    hist_sale_month = hist.loc[(hist.index >= sale_month_start) & (hist.index <= sale_month_finish), price_field]
    sale_date = pd.Timestamp(hist_sale_month.index.values[hist_sale_month.size // 2])
    # sale_price = mean by sale month
    sale_price = hist_sale_month.mean()

    divs = divs.loc[divs.index<=sale_date]
    hist = hist.loc[hist.index<=sale_date]
    div_sum = divs["value"].sum()

    res = hist.join(divs)
    res["buy_date"] = res.index
    res["dividends"] = div_sum - res["value"].fillna(0).cumsum()
    res["real_rate"] = res.apply(lambda row: profit.real_rate(row["buy_date"], row[price_field], sale_date, sale_price, row["dividends"]), axis=1)
    return res[["real_rate"]]

In [25]:
sec_id = "TATN"
sale_year = 2020
sale_month = 1

res = calc_rate(sec_id, sale_year, sale_month, "LEGALCLOSEPRICE")

In [27]:
show_date = pd.Timestamp(sale_year, sale_month, 1) + pd.DateOffset(days=-30)
# res[res.index<show_date].iplot(y=["real_rate"], fill=True, xTitle="Dates", yTitle="Percents", title="Interest income per annum ({})".format(sec_id), dimensions=(1400, 800))